In [2]:
import pandas as pd
import numpy as np
import requests

In [4]:
declarations = pd.read_csv(
    'gatorcain_data/declarations_clean_data.csv'
)
declarations['declarationDate'] = pd.to_datetime(declarations['declarationDate'])

In [6]:
declarations

,disasterNumber,declarationType,declarationDate,declarationTitle,designatedArea
0,106,DR,1960-09-12 00:00:00+00:00,HURRICANE DONNA,Statewide
1,175,DR,1964-09-08 00:00:00+00:00,HURRICANE CLEO,Statewide
2,176,DR,1964-09-10 00:00:00+00:00,HURRICANE DORA,Statewide
3,209,DR,1965-09-14 00:00:00+00:00,HURRICANE BETSY,"Monroe (County),Lee (County),Broward (County),..."
4,252,DR,1968-11-07 00:00:00+00:00,HURRICANE GLADYS,"Charlotte (County),Pinellas (County),Citrus (C..."
5,600,DR,1979-09-13 00:00:00+00:00,HURRICANE FREDERIC,"Bay (County),Walton (County),Escambia (County)..."
6,743,DR,1985-09-12 00:00:00+00:00,HURRICANE ELENA,"Franklin (County),Levy (County),Pinellas (Coun..."
7,756,DR,1985-12-03 00:00:00+00:00,HURRICANE KATE,"Jefferson (County),Franklin (County),Gulf (Cou..."
8,955,DR,1992-08-24 00:00:00+00:00,HURRICANE ANDREW,"Broward (County),Collier (County),Monroe (Coun..."
9,3116,EM,1995-08-03 00:00:00+00:00,HURRICANE ERIN,Statewide


In [7]:
declarations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   disasterNumber    53 non-null     int64              
 1   declarationType   53 non-null     object             
 2   declarationDate   53 non-null     datetime64[ns, UTC]
 3   declarationTitle  53 non-null     object             
 4   designatedArea    53 non-null     object             
dtypes: datetime64[ns, UTC](1), int64(1), object(3)
memory usage: 2.2+ KB
